*** IMPORTANT *** 
This code was written for training purposes. The code is shared only for knowledge sharing purposes. The code should not be used in any production environments.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import os
import glob 
import numpy as np

### Split Video ###

Split the video in to frames and write the files. Original videos are in the `./data` folder. Frames are written to `./data/split_data`.




In [ ]:
def split_data(video_path):
    print(video_path)
    
    capture = cv2.VideoCapture(video_path)
    video_name = video_path.split('/')[-1]
    split_data_path = f'./data/split_data/{video_name[:-4]}'
    
    frameNr = 0
    
    if not os.path.exists('./data/split_data/'+video_name[:-4]):
        os.mkdir('./data/split_data/'+video_name[:-4])
    
    while(True):
        success,frame = capture.read()
        if success:
            isWritten = cv2.imwrite(f'{split_data_path}/frame_{frameNr}.jpg', frame)
            if not isWritten:
                print('Error writing the file!',video_name,frameNr)
        else:
            print("Error reading capture")
            break
        frameNr= frameNr+1
    capture.release()
    

In [ ]:
for mp4_path in glob.glob('./data/*.mp4'):
    split_data(mp4_path)

### XOR Operation ###

Select a reference frame. Do a XOR operation with the reference frame. Removes irrelevant foreground noise.

Note: Refrance frame should be selected carefully. As an example I picked frame_0 as the reference frame.

In [ ]:
def get_orig(spath, videoNr,splitNr=0):
    orig = cv2.imread(f'{spath}{videoNr}/frame_{splitNr}.jpg')
    return orig

def get_compare(spath, videoNr, x):
    compare = cv2.imread(f'{spath}{videoNr}/frame_{x}.jpg')
    return compare

def get_diff(orig, compare):
    diffMask = (orig^compare)
    return diffMask

In [ ]:
spath = './data/split_data/'

In [ ]:

folders = os.listdir(spath)
for folder in folders:
    print(folder)
    
    if os.path.isdir(spath+folder):
        jpg_files = os.listdir(spath+folder)
        
        if not os.path.exists(spath+folder+'/mask_data'):
            os.mkdir(spath+folder+'/mask_data')
            
        for f in jpg_files:
            if not f.endswith('.jpg'):
                continue
            frameNr = f.split('.')[0].split('_')[1]
            fo = get_orig(spath, folder)
            fc = get_compare(spath, folder, frameNr)
            dif = get_diff(fo, fc)
            isWritten = cv2.imwrite(f'{spath}{folder}/mask_data/frame_{frameNr}.jpg', dif)
            if not isWritten:
                print('Error writing XOR file')
    else:
        print('Skipping file '+folder+'. Not a directory')

### Processing the frame ##

Thresholding --> Morphological Thransformation --> Edge detection (auto canny) --> Find contours

Try out diffrent techniques listed in these links to suite your usecase. 

* Thresholding : https://docs.opencv.org/2.4.13.7/doc/tutorials/imgproc/threshold/threshold.html#types-of-thresholding

* Morphological Thransformation : https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html

* Edge detection : https://docs.opencv.org/3.4/da/d22/tutorial_py_canny.html

* Find contours : https://docs.opencv.org/4.x/d4/d73/tutorial_py_contours_begin.html: 


Note: Threshold works better when the image is in grayscale. So after the XOR operation it needs to be converted to gray scale.


In [ ]:
def auto_canny(image, sigma=0.33):
  # compute the median of the single channel pixel intensities
  v = np.median(image)
  # apply automatic Canny edge detection using the computed median
  lower = int(max(0, (1.0 - sigma) * v))
  upper = int(min(255, (1.0 + sigma) * v))
  edged = cv2.Canny(image, lower, upper)
  # return the edged image
  return edged

In [ ]:
def process_frame(img_path, isOrig):
    oimg = cv2.imread(f'{img_path}')
    if not isOrig:
        gray = cv2.cvtColor(oimg, cv2.COLOR_BGR2GRAY)
        (T, thresh) = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    else:
        (T, thresh) = cv2.threshold(oimg, 127, 255, cv2.THRESH_BINARY)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(2,2))
    # print(thresh)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations = 2)
    canny = auto_canny(opening)
    return canny

In [ ]:
def draw_contour(_canny, image):
    im, contours, _ = cv2.findContours(_canny.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cv2.drawContours(image, contours, -1, (0, 0, 255), 2)
    return image

In [ ]:
### Draw the contour on the original video.
### Contour is generated for two instances: (1) using the original video frames and, (2) using the xor operation output frames.

folders = os.listdir(spath)
for folder in folders:
    print(folder)
    
    if os.path.isdir(spath+folder):
        
        if not os.path.exists(spath+folder+'/contour_data'):
            os.mkdir(spath+folder+'/contour_data')
            
        if not os.path.exists(spath+folder+'/mask_contour_data'):
            os.mkdir(spath+folder+'/mask_contour_data')

        jpg_files = os.listdir(spath+folder)

        for f in jpg_files:
            if not f.endswith('.jpg'):
                continue
            frameNr = f.split('.')[0].split('_')[1]

            ac = process_frame(spath+folder+'/'+f, True)
            orig_fimg  = draw_contour(ac, cv2.imread(f'{spath}{folder}/{f}')) 
            isWritten = cv2.imwrite(f'{spath}{folder}/contour_data/frame_{frameNr}.jpg', orig_fimg)
            if not isWritten:
                print('Error writing contour file')

            mask_ac = process_frame(f'{spath}{folder}/mask_data/{f}', False)
            mask_fimg = draw_contour(mask_ac, cv2.imread(f'{spath}{folder}/{f}')) 
            isWritten = cv2.imwrite(f'{spath}{folder}/mask_contour_data/frame_{frameNr}.jpg', mask_fimg)
            if not isWritten:
                print('Error writing contour file')
    else:
        print('Skipping file '+folder+'. Not a directory')

### Generate the final video ###

Combine the frames with contours and output a mp4 file. 

In [ ]:
def output_videos(folder, isOrig):
    
    img_array = []
    for i in range(len(glob.glob(spath+folder+'/contour_data/*.jpg'))):
        if isOrig:
            img = cv2.imread(spath+folder+f'/contour_data/frame_{i}.jpg')
        else: 
            img = cv2.imread(spath+folder+f'/mask_contour_data/frame_{i}.jpg')
        
        height, width, layers = img.shape
        size = (width,height)
        img_array.append(img)
        
    if isOrig:
        out = cv2.VideoWriter(spath+folder+'/contour_data/contour_data.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
    else: 
        out = cv2.VideoWriter(spath+folder+'/mask_contour_data/mask_contour_data.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)        

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
        

In [ ]:
folders = os.listdir(spath)
for folder in folders:
    output_videos(folder, False)
    output_videos(folder, True)